## 직무 기술서 데이터 불러오기

In [1]:
import pandas as pd
import json
import os
from dotenv import load_dotenv

# .env 파일에서 OpenAI API 키 로드 및 환경 변수 설정
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY", None)
if openai_api_key is None:
    raise ValueError(".env 파일에 'OPENAI_API_KEY'가 없습니다. 변수 설정을 확인하세요.")
os.environ["OPENAI_API_KEY"] = openai_api_key

# 직무기술서 데이터 로드 (JSON → DataFrame)
json_path = "C:/workspace/Final_project/backend-model/AI_Lab/data/job_description.json"
with open(json_path, encoding="utf-8") as f:
    data = json.load(f)
df = pd.DataFrame(data)
df.head()

# 채용공고 데이터 로드 (회사별 JSON → DataFrame)
company_name = "kakao"
jobs_json_path = f"C:/workspace/Final_project/backend-model/AI_Lab/data/{company_name}_jobs.json"
with open(jobs_json_path, encoding="utf-8") as f:
    jobs_data = json.load(f)
jobs_df = pd.DataFrame(jobs_data)
jobs_df.drop('html', axis=1, inplace=True)
jobs_df.head()

,title,company,location,employment_type,experience,crawl_date,posted_date,expired_date,description,url,meta_data,screenshots,skill_set
0,네트워크 보안 담당자 (보안),카카오,판교,정규직,경력 2~10년,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14247?skillSe...,"{'job_category': '보안', 'preferred_qualificatio...",{'combined': 'data\kakao_screenshots\kakao_job...,[협업]
1,Machine Learning Engineer (Search/ML/LLM) (신입),카카오,판교,정규직,신입,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14286?skillSe...,"{'job_category': 'AI/ML', 'preferred_qualifica...",{'combined': 'data\kakao_screenshots\kakao_job...,"[ElasticSearch, LLM, Machine Learning, NLP, Py..."
2,Data Engineer (경력),카카오,판교,정규직,경력 5년 이상,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14271?skillSe...,"{'job_category': '기타', 'preferred_qualificatio...",{'combined': 'data\kakao_screenshots\kakao_job...,"[Java, Python, 분석, 협업]"
3,보이스톡 / 페이스톡 Server 엔지니어 (경력),카카오,판교,정규직,경력 2년 이상,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14172?skillSe...,"{'job_category': 'IT/개발', 'preferred_qualifica...",{'combined': 'data\kakao_screenshots\kakao_job...,"[node.js, 분석, 전략 수립, 통신, 협업]"
4,비즈앱 iOS 개발자(경력),카카오,판교,정규직,경력 5년 이상,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14169?skillSe...,"{'job_category': 'IT/개발', 'preferred_qualifica...",{'combined': 'data\kakao_screenshots\kakao_job...,"[Git, 협업]"


In [2]:

# 하이브리드 검색(Retriever) 관련 라이브러리
from langchain_community.vectorstores import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from typing import List

# 여러 회사별 채용공고 파일을 효율적으로 한 번에 로드
job_postings = []
job_paths = [
    "C:/workspace/Final_project/backend-model/AI_Lab/data/kakao_jobs.json",
    "C:/workspace/Final_project/backend-model/AI_Lab/data/woowahan_jobs.json",
    "C:/workspace/Final_project/backend-model/AI_Lab/data/hanwha_jobs.json",
    "C:/workspace/Final_project/backend-model/AI_Lab/data/line_jobs.json",
    "C:/workspace/Final_project/backend-model/AI_Lab/data/naver_jobs.json",
]

for path in job_paths:
    try:
        with open(path, 'r', encoding='utf-8') as f:
            job_postings.extend(json.load(f))
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {path}")
    except json.JSONDecodeError:
        print(f"JSON decode 오류: {path}")

print(f"총 채용공고 수: {len(job_postings)}")


총 채용공고 수: 217


In [ ]:
job_postings[-1]

{'title': 'Omni-Foundation Model Pathfinding Research Scientist',
 'company': 'NAVER Cloud',
 'location': '경기 성남시 분당구 불정로 6 (그린팩토리)',
 'employment_type': '정규',
 'experience': '경력',
 'crawl_date': '2025-11-07',
 'posted_date': '2025-11-03',
 'expired_date': '2025-11-21',
 'description': "[NAVER Cloud] Omni-Foundation Model Pathfinding Research Scientist (경력)\n모집 부서\nNAVER Cloud \n모집 분야\nTech \n모집 분야\nAI/ML \n모집 경력\n경력 \n근로 조건\n정규\n모집 기간\n2025.11.03 ~ 2025.11.21 (17:00)\nShare Save\n지원하기\n\n\ufeff\n\n\n\n부서소개\n\n\ufeff저희 부서는 HyperCLOVA X 등 소버린 파운데이션 기술의 경쟁력을 강화하기 위해, 최신 기술 동향을 사내외로 전파하고 원천 AI 기술을 연구·개발합니다. 또한 산학 협력과 논문 발표 등을 통해 글로벌 AI 동향에 발맞춘 연구를 수행하고 있습니다. 더불어 자기주도적으로 최신 기술 동향을 파악하여 조직의 미래 가치에 기여하며, 조직 기술의 강점과 단점을 파악하여 나아가야 할 방향을 제언합니다.\n\n예를 들어, HyperCLOVA X와 같은 자체 한국어 기반 모델의 기술 고도화에 기여하거나 연구 분야의 Low hanging fruit을 구별하고 Minimum Viable Product(MVP)를 통한 initial value를 제공할 수 있습니다. 글로벌 AI 동향에 맞춰 선행 연구를 개발하고, 이를 Top-tier 해외 학회에 발표하는 등의 HyperCLOVA의 기술 우위력을 적극적으로 홍보하는 역할을 수행합니다.\n\n담당업무\n\n\u

In [ ]:
# 채용공고 리스트 → LangChain Document로 변환
documents = []
for job in job_postings:
    skill_set = job.get('skill_set', [])
    skill_set_text = ", ".join(skill_set) if isinstance(skill_set, list) else str(skill_set)
    required_skills_text = ""
    job_category = ""
    
    # meta_data 안전하게 처리 (딕셔너리인지 확인)
    meta_data = job.get('meta_data', {})
    if isinstance(meta_data, dict) and meta_data:
        if 'required_skills' in meta_data and meta_data['required_skills']:
            required_skills_text = ", ".join(meta_data['required_skills']) if isinstance(meta_data['required_skills'], list) else str(meta_data['required_skills'])
        if 'job_category' in meta_data:
            job_category = meta_data['job_category']
    
    content = f"""
    제목: {job.get('title', '')}
    회사: {job.get('company', '')}
    직무내용: {job.get('description', '')[:500] if job.get('description') else ''}
    업무분야: {job_category}
    필요 스킬: {required_skills_text}
    스킬셋: {skill_set_text}
    경력: {job.get('experience', '')}
    """
    metadata = {
        "title": job.get('title', ''),
        "company": job.get('company', ''),                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
        "url": job.get('url', ''),
        "job_category": job_category,
        "experience": job.get('experience', ''),
    }
    documents.append(Document(page_content=content.strip(), metadata=metadata))
print(f"Document 변환 완료: {len(documents)}개")


In [ ]:

# 의미론적 검색(임베딩 기반)용 ChromaDB 생성 및 Retriever 준비
print("ChromaDB 벡터 스토어 생성 중...")
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="job_postings"
)
chroma_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
print("ChromaDB 생성 완료")

# 키워드 기반 검색(BM25) Retriever 준비
print("BM25 Retriever 생성 중...")
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 10
print("BM25 Retriever 생성 완료")

# 하이브리드(앙상블) Retriever 정의
class CustomEnsembleRetriever(BaseRetriever):
    """여러 retriever를 결합하여 결과를 종합(가중치 반영)"""
    retrievers: List[BaseRetriever]
    weights: List[float]
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        all_results = []
        for retriever, weight in zip(self.retrievers, self.weights):
            results = retriever.invoke(query)
            for doc in results:
                doc.metadata['ensemble_score'] = weight
                all_results.append(doc)
        # 내용 일부(PageContent) 기준 중복 제거 및 가중치 합산
        unique_docs = {}
        for doc in all_results:
            doc_id = doc.page_content[:100]
            if doc_id in unique_docs:
                unique_docs[doc_id].metadata['ensemble_score'] += doc.metadata.get('ensemble_score', 0)
            else:
                unique_docs[doc_id] = doc
        sorted_docs = sorted(
            unique_docs.values(), 
            key=lambda x: x.metadata.get('ensemble_score', 0), 
            reverse=True
        )
        return sorted_docs

# 하이브리드 검색기 생성 (임베딩: 30%, BM25: 70%)
ensemble_retriever = CustomEnsembleRetriever(
    retrievers=[chroma_retriever, bm25_retriever],
    weights=[0.3, 0.7]
)
print("Ensemble Retriever 생성 완료")

# 직무기술서 갯수 및 샘플 출력
print(f"직무기술서 데이터: {len(df)}개")
df.head()

def search_similar_jobs(industry, skill_set, top_k=5):
    """
    직무기술서의 industry와 skill_set을 기반으로 유사 채용공고 검색 (최대 top_k개 반환)
    """
    skill_set_str = " ".join(skill_set) if isinstance(skill_set, list) else str(skill_set)
    query = f"{industry} {skill_set_str}"
    results = ensemble_retriever.invoke(query)
    return results[:top_k]

# 결과 저장용 변수
from datetime import datetime
all_results = []

# 직무기술서 전체에 대해 유사 채용공고 검색
print("\n" + "="*100)
print("전체 직무기술서에 대한 채용공고 매칭 시작...")
print("="*100)

for idx, row in df.iterrows():
    industry = row['industry']
    skill_set = row['공통_skill_set']
    skill_set += row['skill_set']
    job_title = row['직무']
    
    # 검색 실행
    similar_jobs = search_similar_jobs(industry, skill_set, top_k=5)
    
    # 결과 저장
    result_item = {
        'job_title': job_title,
        'industry': industry,
        'skill_set': skill_set[:5] if isinstance(skill_set, list) else str(skill_set)[:100],
        'matched_jobs': []
    }
    
    for i, job in enumerate(similar_jobs, 1):
        result_item['matched_jobs'].append({
            'rank': i,
            'title': job.metadata['title'],
            'company': job.metadata['company'],
            'job_category': job.metadata.get('job_category', 'N/A'),
            'experience': job.metadata.get('experience', 'N/A'),
            'url': job.metadata['url'],
            'score': job.metadata.get('ensemble_score', 0)
        })
    
    all_results.append(result_item)
    
    # 진행상황 출력
    if (idx + 1) % 10 == 0:
        print(f"진행 중... {idx + 1}/{len(df)} 완료")

print(f"\n매칭 완료! 총 {len(all_results)}개 직무기술서 처리")

# TXT 파일로 저장
def save_results_to_txt(results, filename):
    """검색 결과를 TXT 파일로 저장"""
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("="*100 + "\n")
        f.write("직무기술서 - 채용공고 매칭 결과\n")
        f.write(f"생성일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"총 직무기술서 수: {len(results)}개\n")
        f.write(f"총 채용공고 수: {len(job_postings)}개\n")
        f.write("="*100 + "\n\n")
        
        for idx, result in enumerate(results, 1):
            f.write(f"\n{'='*100}\n")
            f.write(f"[{idx}] 직무: {result['job_title']}\n")
            f.write(f"{'='*100}\n")
            f.write(f"Industry: {result['industry']}\n")
            f.write(f"Skill Set (샘플): {result['skill_set']}\n")
            f.write(f"\n{'추천 채용공고 Top 5':-^90}\n\n")
            
            for job in result['matched_jobs']:
                f.write(f"{job['rank']}. {job['title']}\n")
                f.write(f"   회사: {job['company']}\n")
                f.write(f"   업무분야: {job['job_category']}\n")
                f.write(f"   경력: {job['experience']}\n")
                f.write(f"   매칭점수: {job['score']:.2f}\n")
                f.write(f"   URL: {job['url']}\n\n")
    
    print(f"✅ TXT 파일 저장 완료: {filename}")

# PDF 파일로 저장 (reportlab 사용)
def save_results_to_pdf(results, filename):
    """검색 결과를 PDF 파일로 저장 (한글 지원)"""
    try:
        from reportlab.lib.pagesizes import A4, letter
        from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
        from reportlab.lib.units import inch
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
        from reportlab.pdfbase import pdfmetrics
        from reportlab.pdfbase.ttfonts import TTFont
        from reportlab.lib.enums import TA_LEFT, TA_CENTER
        
        # 한글 폰트 등록 시도
        try:
            # Windows 기본 한글 폰트
            pdfmetrics.registerFont(TTFont('Malgun', 'malgun.ttf'))
            font_name = 'Malgun'
        except:
            try:
                pdfmetrics.registerFont(TTFont('Gulim', 'gulim.ttf'))
                font_name = 'Gulim'
            except:
                print("⚠️ 한글 폰트를 찾을 수 없습니다. 영문 폰트로 대체합니다.")
                font_name = 'Helvetica'
        
        # PDF 생성
        doc = SimpleDocTemplate(filename, pagesize=A4)
        story = []
        
        # 스타일 정의
        styles = getSampleStyleSheet()
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontName=font_name,
            fontSize=16,
            alignment=TA_CENTER
        )
        heading_style = ParagraphStyle(
            'CustomHeading',
            parent=styles['Heading2'],
            fontName=font_name,
            fontSize=12
        )
        normal_style = ParagraphStyle(
            'CustomNormal',
            parent=styles['Normal'],
            fontName=font_name,
            fontSize=9
        )
        
        # 제목
        story.append(Paragraph("직무기술서 - 채용공고 매칭 결과", title_style))
        story.append(Spacer(1, 0.2*inch))
        story.append(Paragraph(f"생성일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", normal_style))
        story.append(Paragraph(f"총 직무기술서 수: {len(results)}개 | 총 채용공고 수: {len(job_postings)}개", normal_style))
        story.append(Spacer(1, 0.3*inch))
        
        # 각 결과 추가
        for idx, result in enumerate(results, 1):
            story.append(Paragraph(f"[{idx}] 직무: {result['job_title']}", heading_style))
            story.append(Paragraph(f"Industry: {result['industry']}", normal_style))
            story.append(Paragraph(f"Skill Set: {str(result['skill_set'])[:100]}", normal_style))
            story.append(Spacer(1, 0.1*inch))
            
            story.append(Paragraph("추천 채용공고 Top 5", heading_style))
            for job in result['matched_jobs']:
                story.append(Paragraph(
                    f"{job['rank']}. {job['title']} - {job['company']}", 
                    normal_style
                ))
                story.append(Paragraph(
                    f"   업무분야: {job['job_category']} | 경력: {job['experience']} | 점수: {job['score']:.2f}", 
                    normal_style
                ))
                story.append(Paragraph(f"   URL: {job['url']}", normal_style))
                story.append(Spacer(1, 0.05*inch))
            
            story.append(Spacer(1, 0.2*inch))
            
            # 10개마다 페이지 나누기
            if idx % 10 == 0:
                story.append(PageBreak())
        
        # PDF 빌드
        doc.build(story)
        print(f"✅ PDF 파일 저장 완료: {filename}")
        
    except ImportError:
        print("⚠️ reportlab 라이브러리가 설치되어 있지 않습니다.")
        print("   'pip install reportlab' 명령으로 설치 후 다시 시도하세요.")
    except Exception as e:
        print(f"⚠️ PDF 생성 중 오류 발생: {str(e)}")

# 결과 파일 저장
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = "C:/workspace/Final_project/backend-model/AI_Lab/output"
os.makedirs(output_dir, exist_ok=True)

txt_filename = f"{output_dir}/job_matching_results_{timestamp}.txt"
pdf_filename = f"{output_dir}/job_matching_results_{timestamp}.pdf"

# TXT 저장
save_results_to_txt(all_results, txt_filename)

# PDF 저장
save_results_to_pdf(all_results, pdf_filename)

print("\n" + "="*100)
print("결과 파일 저장 완료!")
print(f"📄 TXT 파일: {txt_filename}")
print(f"📄 PDF 파일: {pdf_filename}")
print("="*100)

# 처음 3개 결과만 콘솔에 출력
print("\n\n[검색 결과 샘플 - 처음 3개]")
for idx, result in enumerate(all_results[:3], 1):
    print(f"\n{'='*100}")
    print(f"[{idx}] 직무: {result['job_title']}")
    print(f"Industry: {result['industry']}")
    print(f"{'='*100}")
    for job in result['matched_jobs']:
        print(f"\n{job['rank']}. {job['title']}")
        print(f"   회사: {job['company']}")
        print(f"   업무분야: {job['job_category']}")
        print(f"   경력: {job['experience']}")
        print(f"   매칭점수: {job['score']:.2f}")
        print(f"   URL: {job['url']}")


## 채용공고 데이터 불러오기

In [2]:
# 사용할 회사명을 변수로 지정 (예: "kakao" 또는 "woowahan" 등)
company_name = "kakao"

# 파일명 생성
jobs_json_path = f"C:/workspace/Final_project/backend-model/AI_Lab/data/{company_name}_jobs.json"

# JSON 파일 불러오기
with open(jobs_json_path, encoding="utf-8") as f:
    jobs_data = json.load(f)

# 데이터프레임으로 변환 (만약 리스트 구조라면)
jobs_df = pd.DataFrame(jobs_data)

# 미리보기
jobs_df.drop('html', axis=1,inplace=True)


In [8]:
jobs_df.head()

,title,company,location,employment_type,experience,crawl_date,posted_date,expired_date,description,url,meta_data,screenshots,skill_set
0,네트워크 보안 담당자 (보안),카카오,판교,정규직,경력 2~10년,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14247?skillSe...,"{'job_category': '보안', 'preferred_qualificatio...",{'combined': 'data\kakao_screenshots\kakao_job...,[협업]
1,Machine Learning Engineer (Search/ML/LLM) (신입),카카오,판교,정규직,신입,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14286?skillSe...,"{'job_category': 'AI/ML', 'preferred_qualifica...",{'combined': 'data\kakao_screenshots\kakao_job...,"[ElasticSearch, LLM, Machine Learning, NLP, Py..."
2,Data Engineer (경력),카카오,판교,정규직,경력 5년 이상,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14271?skillSe...,"{'job_category': '기타', 'preferred_qualificatio...",{'combined': 'data\kakao_screenshots\kakao_job...,"[Java, Python, 분석, 협업]"
3,보이스톡 / 페이스톡 Server 엔지니어 (경력),카카오,판교,정규직,경력 2년 이상,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14172?skillSe...,"{'job_category': 'IT/개발', 'preferred_qualifica...",{'combined': 'data\kakao_screenshots\kakao_job...,"[node.js, 분석, 전략 수립, 통신, 협업]"
4,비즈앱 iOS 개발자(경력),카카오,판교,정규직,경력 5년 이상,2025-11-07,2025-11-07,None,◆ 직원 유형\n정규직\n\n\n◆ 영입인원\n0 명\n\n\n◆ 조직소개\n\n\...,https://careers.kakao.com/jobs/P-14169?skillSe...,"{'job_category': 'IT/개발', 'preferred_qualifica...",{'combined': 'data\kakao_screenshots\kakao_job...,"[Git, 협업]"


In [4]:
df_solution_dev_erp_scm = df[(df['직무'] == 'Solution Development') & (df['industry'] == 'ERP_SCM (Supply Chain Management)')]
df_solution_dev_erp_scm


,직무,industry,공통_skill_set,skill_set
7,Solution Development,ERP_SCM (Supply Chain Management),"[고객요구사항분석, Solution별기능, 솔루션Architecture 설계방법론,...","Order to cash, Procure to pay, Supply Chain, P..."


In [5]:
df_solution_dev_erp_scm['skill_set'].values

array(['Order to cash, Procure to pay, Supply Chain, Plan to produce 등 기업 End to End Process, ERPMM, SD, WM, TD, PP, PM, QM 모듈의 Configuration 및 Program 활용, Plant, Storage location, Sales / Purchase Org. 등 구성방법, Vendor, Customer, Material, Product, BOM 등 SC Data 구조, Data 이관전략수립.'],
      dtype=object)

## 하이브리드 Retriever

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
import pandas as pd
import json

# 1. 채용공고 데이터 로드
with open('job_postings.json', 'r', encoding='utf-8') as f:
    job_postings = json.load(f)

# 2. Document 객체로 변환
documents = []
for job in job_postings:
    # 검색에 사용할 텍스트 구성
    content = f"""
    제목: {job['title']}
    회사: {job['company']}
    직무내용: {job['description']}
    업무분야: {job.get('meta_data', {}).get('job_category', '')}
    """
    
    metadata = {
        "title": job['title'],
        "company": job['company'],
        "url": job['url'],
        "job_category": job.get('meta_data', {}).get('job_category', '')
    }
    
    documents.append(Document(page_content=content.strip(), metadata=metadata))

# 3. ChromaDB retriever 생성
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="job_postings"
)
chroma_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 10}
)

# 4. BM25 retriever 생성
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 10

# 5. Ensemble Retriever 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, bm25_retriever],
    weights=[0.6, 0.4]  # 의미론적 검색에 더 가중치
)

# 6. 직무기술서 데이터로 검색
jd_df = pd.read_csv('job_description.csv')

def search_similar_jobs(industry, skill_set, top_k=5):
    """
    직무기술서의 industry와 skill_set으로 유사한 채용공고 검색
    """
    # 검색 쿼리 구성
    query = f"{industry} {skill_set}"
    
    # 검색 실행
    results = ensemble_retriever.get_relevant_documents(query)
    
    return results[:top_k]

# 7. 각 직무기술서에 대해 유사한 채용공고 찾기
for idx, row in jd_df.iterrows():
    industry = row['industry']
    skill_set = row['공통_skill_set']
    
    print(f"\n직무: {row['직무']}")
    print(f"Industry: {industry}")
    print(f"Skill Set: {skill_set}")
    print("=" * 80)
    
    similar_jobs = search_similar_jobs(industry, skill_set, top_k=5)
    
    for i, job in enumerate(similar_jobs, 1):
        print(f"{i}. {job.metadata['title']}")
        print(f"   회사: {job.metadata['company']}")
        print(f"   URL: {job.metadata['url']}")
        print()

O
r
d
e
r
 
t
o
 
c
a
s
h
,
 
P
r
o
c
u
r
e
 
t
o
 
p
a
y
,
 
S
u
p
p
l
y
 
C
h
a
i
n
,
 
P
l
a
n
 
t
o
 
p
r
o
d
u
c
e
 
등
 
기
업
 
E
n
d
 
t
o
 
E
n
d
 
P
r
o
c
e
s
s
,
 
E
R
P
M
M
,
 
S
D
,
 
W
M
,
 
T
D
,
 
P
P
,
 
P
M
,
 
Q
M
 
모
듈
의
 
C
o
n
f
i
g
u
r
a
t
i
o
n
 
및
 
P
r
o
g
r
a
m
 
활
용
,
 
P
l
a
n
t
,
 
S
t
o
r
a
g
e
 
l
o
c
a
t
i
o
n
,
 
S
a
l
e
s
 
/
 
P
u
r
c
h
a
s
e
 
O
r
g
.
 
등
 
구
성
방
법
,
 
V
e
n
d
o
r
,
 
C
u
s
t
o
m
e
r
,
 
M
a
t
e
r
i
a
l
,
 
P
r
o
d
u
c
t
,
 
B
O
M
 
등
 
S
C
 
D
a
t
a
 
구
조
,
 
D
a
t
a
 
이
관
전
략
수
립
.


In [7]:
len(df_solution_dev_erp_scm['공통_skill_set'][0])

KeyError: 0